In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Strategy 2 pipeline:
1) Divide video into 10s scenes
2) Extract 1–2 frames per chunk
3) Generate global description: video type + video description
4) First chunk uses (global desc + prompt + frames) → Description/OCR/Entities/Chronological actions
5) Global context is updated for every scene (frequencies, confront facts)
6) Save results in files (CSV + metadata TXT + context JSON)
"""

import os
import re
import csv
import json
import math
import time
import datetime as dt
import logging
import sys
from threading import Thread
from typing import Dict, Any, List, Tuple

import numpy as np
import torch
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from PIL import Image
from decord import VideoReader, cpu
from transformers import AutoModel, AutoTokenizer, TextIteratorStreamer

In [2]:
# ----------------------------
# Config
# ----------------------------
VIDEO_PATH = "documentary_480p.mp4"  # change as needed
SCENE_SECONDS = 10
GLOBAL_SAMPLE_SEGMENTS = 16
GLOBAL_FRAMES_PER_SEGMENT = 1     # 1 frame per global segment
SCENE_SEGMENTS = 10
SCENE_FRAMES_PER_SEGMENT = 1      # 1–2 frames per chunk -> here we take 2
TILE_MAX_NUM = 1
INPUT_SIZE = 448
USE_CUDA = torch.cuda.is_available()

# Decoding settings for safer generations
GEN_MAX_NEW_TOKENS = 768
GEN_TEMPERATURE = 0.7
GEN_TOP_P = 0.9
GEN_TOP_K = 50
GEN_REP_PENALTY = 1.15
GEN_NO_REPEAT_NGRAM = 6

In [3]:
# ----------------------------
# Logging (verbose)
# ----------------------------
def _setup_logger(name: str = "video_rag", level: int = logging.INFO) -> logging.Logger:
    logger = logging.getLogger(name)
    if logger.handlers:
        return logger
    logger.setLevel(level)
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s | %(levelname)-7s | %(message)s", "%Y-%m-%d %H:%M:%S")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

LOGGER = _setup_logger()

In [4]:
# ----------------------------
# Model init (InternVL 3.5-8B)
# ----------------------------
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

path = "OpenGVLab/InternVL3_5-8B"
LOGGER.info("Loading model...")
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map="auto"
).eval()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)
LOGGER.info("Model loaded.")

2025-09-13 15:39:47 | INFO    | Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-09-13 15:39:53 | INFO    | Model loaded.


In [10]:
# ----------------------------
# Image / Video utilities
# ----------------------------
def build_transform(input_size):
    return T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf'); best_ratio = (1, 1); area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff; best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1)
        for i in range(1, n + 1) for j in range(1, n + 1)
        if i * j <= max_num and i * j >= min_num
    )
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        processed_images.append(resized_img.crop(box))
    if use_thumbnail and len(processed_images) != 1:
        processed_images.append(image.resize((image_size, image_size)))
    return processed_images

def build_transform_and_stack(frames_as_pil, input_size=448, max_num=1):
    transform = build_transform(input_size=input_size)
    tiles = []
    for img in frames_as_pil:
        img_tiles = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        tile_tensor = [transform(tile) for tile in img_tiles]
        tile_tensor = torch.stack(tile_tensor)
        tiles.append(tile_tensor)
    pixel_values = torch.cat(tiles)
    return pixel_values

def seconds_to_tc(t):
    hours = int(t // 3600); minutes = int((t % 3600) // 60); seconds = t % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:06.3f}"

def get_frame_indices(bound, fps, max_frame, first_idx=0, num_segments=32, frames_per_segment=1):
    """
    Evenly spaced segments; optionally take multiple frames per segment.
    Deterministic (no randomness).
    """
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    if end_idx <= start_idx:
        end_idx = min(start_idx + max(1, int(fps)), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = []
    for idx in range(num_segments):
        seg_start = start_idx + seg_size * idx
        # evenly spread frames inside the segment
        for j in range(frames_per_segment):
            pos = seg_start + (seg_size * (j + 0.5) / frames_per_segment)
            frame_indices.append(int(pos))
    frame_indices = np.clip(frame_indices, start_idx, end_idx)
    return frame_indices

def load_video_pixel_values(video_path, bound=None, input_size=448, max_num=1,
                            num_segments=32, frames_per_segment=1):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())
    idxs = get_frame_indices(bound, fps, max_frame, first_idx=0,
                             num_segments=num_segments, frames_per_segment=frames_per_segment)
    frames_as_pil = [Image.fromarray(vr[int(i)].asnumpy()).convert('RGB') for i in idxs]
    pixel_values = build_transform_and_stack(frames_as_pil, input_size=input_size, max_num=max_num)
    width, height = vr[0].shape[1], vr[0].shape[0]
    return pixel_values, len(idxs), fps, width, height

# ----------------------------
# Generation helpers (Robust, Verbose, No duplicate use_cache)
# ----------------------------
import sys, time, re
from queue import Empty as _QueueEmpty
from threading import Thread

_REPEAT_CHAR_RE = re.compile(r"(.)\1{15,}", re.DOTALL)
_REPEAT_CHUNK_RE = re.compile(r"(.{3,60})\1{3,}", re.DOTALL)

def _looks_degenerate(s: str) -> bool:
    tail = s[-2000:]
    return bool(_REPEAT_CHAR_RE.search(tail) or _REPEAT_CHUNK_RE.search(tail))

def _trim_degenerate_tail(s: str) -> str:
    for cut in range(len(s), max(len(s) - 2000, 0), -1):
        tail = s[:cut]
        if not _looks_degenerate(tail):
            return tail.rstrip()
    return s.rstrip()

def chat_with_images(pixel_values, question, timeout=30):
    """
    Streamed verbose generation with safeguards against degenerate repetition and
    a robust thread/streamer loop that won't hang if model.chat raises.
    """
    t0 = time.perf_counter()
    if USE_CUDA:
        pixel_values = pixel_values.to(torch.bfloat16).cuda()
        LOGGER.info(f"chat_with_images | CUDA | tensor={tuple(pixel_values.shape)}")
    else:
        pixel_values = pixel_values.to(torch.bfloat16)
        LOGGER.info(f"chat_with_images | CPU | tensor={tuple(pixel_values.shape)}")

    question = question.strip() + (
        "\n\nCONSTRAINTS: Be concise. Do not repeat identical tokens/characters. "
        "If text is unreadable, write 'unknown' once."
    )
    LOGGER.info(f"Prompt length: {len(question)} chars")

    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True, timeout=0.2)

    # IMPORTANT: do NOT pass use_cache here; InternVL passes it internally.
    generation_config = dict(
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        do_sample=True,
        temperature=GEN_TEMPERATURE,
        top_p=GEN_TOP_P,
        top_k=GEN_TOP_K,
        repetition_penalty=GEN_REP_PENALTY,
        no_repeat_ngram_size=GEN_NO_REPEAT_NGRAM,
        renormalize_logits=True,
        streamer=streamer,
        # eos_token_id left to model defaults; avoid duplicating args InternVL may set.
    )

    worker_error = {"exc": None}

    def _run_chat():
        try:
            model.chat(
                tokenizer=tokenizer,
                pixel_values=pixel_values,
                question=question,
                history=None,
                return_history=False,
                generation_config=generation_config,
            )
        except Exception as e:
            worker_error["exc"] = e
            LOGGER.exception("model.chat raised an exception")

    thread = Thread(target=_run_chat, daemon=True)
    thread.start()

    chunks = []
    LOGGER.info(">>> Generation started...")
    while True:
        try:
            new_text = next(streamer)  # may raise StopIteration when done
            # Respect model separator if present
            if new_text == getattr(model, "conv_template", None).sep if hasattr(model, "conv_template") else False:
                LOGGER.debug("Separator reached, stopping.")
                break

            chunks.append(new_text)
            sys.stdout.write(new_text); sys.stdout.flush()

            if _looks_degenerate("".join(chunks)):
                LOGGER.warning("\n>>> Degenerate repetition detected — stopping early.")
                break

        except StopIteration:
            # Streamer signaled completion
            break
        except _QueueEmpty:
            # No token right now; check if worker died/crashed or finished
            if not thread.is_alive():
                # Worker finished (possibly with error) and no more tokens are coming
                break
            # else: keep waiting for tokens
            continue

    # Join the worker to surface any exception
    thread.join(timeout=1.0)
    if worker_error["exc"] is not None:
        # Surface a compact error into the text so upstream can still continue gracefully
        err_msg = f"[MODEL_ERROR] {type(worker_error['exc']).__name__}: {worker_error['exc']}"
        LOGGER.error(err_msg)
        if chunks:
            chunks.append("\n" + err_msg)
        else:
            return err_msg

    result = "".join(chunks).strip()
    if _looks_degenerate(result):
        clean = _trim_degenerate_tail(result)
        if clean != result:
            LOGGER.warning(">>> Trimmed trailing degenerate text.")
            result = clean + " [TRUNCATED]"

    LOGGER.info(f"\n>>> Generation finished in {time.perf_counter() - t0:.2f}s "
                f"({len(result)} chars).")
    return result

# ----------------------------
# Context state & parsing
# ----------------------------
def init_context_state() -> Dict[str, Any]:
    return {
        "video_type": None,
        "summary": None,
        "style": None,
        "entities_freq": {},  # name_or_value -> count
        "entity_types": {},   # name_or_value -> last type seen
        "actions_freq": {},   # action -> count
        "tags_freq": {},      # tag -> count
        "ocr_vocab": {},      # ocr token -> count
        "scenes_seen": 0
    }

def normalize_token(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip()).lower()

def parse_scene_card(card: str) -> Dict[str, Any]:
    """
    Lightweight parser for the Scene Card sections.
    Robust to minor formatting variations.
    """
    sections = {
        "Meta": r"Meta:\s*(.*?)\n\n",
        "Description": r"Description:\s*(.*?)\n\n",
        "OnScreenText": r"OnScreenText:\s*(.*?)\n\n",
        "Entities": r"Entities:\s*(.*?)\n\n",
        "Tags": r"Tags:\s*(.*?)\n\n",
        "Shot": r"Shot:\s*(.*?)\n\n",
        "Confidence": r"Confidence:\s*(.*)$",
    }
    out = {}
    for key, pat in sections.items():
        m = re.search(pat, card, re.DOTALL | re.IGNORECASE)
        out[key] = m.group(1).strip() if m else ""
    return out

def parse_bullets(block: str) -> List[str]:
    items = []
    for line in block.splitlines():
        line = line.strip()
        if re.match(r"^[-•] ", line) or re.match(r"^\d+\.", line):
            items.append(re.sub(r"^[-•]\s*", "", line))
        elif line.startswith("  - "):
            items.append(line[4:])
        elif line.startswith("- "):
            items.append(line[2:])
    return [s.strip() for s in items if s.strip()]

def update_context_from_scene(state: Dict[str, Any], card_text: str, global_text_hint: str = ""):
    parsed = parse_scene_card(card_text)

    # Update OCR vocabulary
    ocr_block = parsed.get("OnScreenText", "")
    ocr_lines = [re.sub(r"^-\s*", "", l.strip()) for l in ocr_block.splitlines() if l.strip().startswith("-")]
    for ln in ocr_lines:
        for tok in re.findall(r"[A-Za-z0-9:#@\-_/]+", ln):
            nt = normalize_token(tok)
            state["ocr_vocab"][nt] = state["ocr_vocab"].get(nt, 0) + 1

    # Update actions
    desc_block = parsed.get("Description", "")
    # Extract "actions_chronological" sub-bullets
    act_match = re.search(r"actions_chronological:\s*(.*)", desc_block, re.DOTALL | re.IGNORECASE)
    if act_match:
        acts = parse_bullets(act_match.group(1))
        for a in acts:
            na = normalize_token(re.sub(r"^t≈\s*[^:]+:\s*", "", a))
            if na:
                state["actions_freq"][na] = state["actions_freq"].get(na, 0) + 1

    # Update entities
    ent_block = parsed.get("Entities", "")
    ent_items = []
    # naive parse: lines with "type:" or "name_or_value:"
    current = {}
    for line in ent_block.splitlines():
        l = line.strip()
        if l.startswith("- ") and "type:" in l:
            if current:
                ent_items.append(current); current = {}
            current = {"type": l.split("type:", 1)[1].strip()}
        elif "name_or_value:" in l:
            current["name_or_value"] = l.split("name_or_value:", 1)[1].strip()
        elif "attributes:" in l:
            attrs = re.findall(r"\[(.*)\]", l)
            current["attributes"] = attrs[0].split(",") if attrs else []
        elif l.startswith("- ") and current:
            # new bullet possibly starts; push previous
            ent_items.append(current); current = {}
    if current:
        ent_items.append(current)

    for ent in ent_items:
        name = normalize_token(ent.get("name_or_value", ""))
        typ = normalize_token(ent.get("type", ""))
        if name:
            state["entities_freq"][name] = state["entities_freq"].get(name, 0) + 1
            if typ:
                state["entity_types"][name] = typ

    # Update tags
    tags_block = parsed.get("Tags", "")
    # lines like: - scene_tags: [tag1, tag2, ...]
    for line in tags_block.splitlines():
        l = line.strip()
        m = re.search(r"scene_tags:\s*\[(.*)\]", l, re.IGNORECASE)
        if m:
            tags = [normalize_token(x) for x in m.group(1).split(",")]
            for t in tags:
                t = t.strip()
                if t:
                    state["tags_freq"][t] = state["tags_freq"].get(t, 0) + 1

    state["scenes_seen"] += 1

def confront_facts(state: Dict[str, Any], global_text: str) -> Dict[str, Any]:
    """
    Produce a simple, updated global context from current frequencies + global_text hints.
    """
    # Basic heuristics: top entities/actions/tags
    def top_k(d, k=10):
        return sorted(d.items(), key=lambda x: (-x[1], x[0]))[:k]

    top_entities = top_k(state["entities_freq"], 15)
    top_actions = top_k(state["actions_freq"], 15)
    top_tags = top_k(state["tags_freq"], 15)

    # Resolve a "likely_type" from global_text (very light heuristic)
    likely_type = None
    m = re.search(r"\[TYPE\]\s*(.*)", global_text, re.IGNORECASE)
    if m:
        likely_type = m.group(1).strip()

    # Build text block to feed next scene
    ctx_lines = []
    if likely_type:
        ctx_lines.append(f"Likely video type: {likely_type}")
    if state["summary"]:
        ctx_lines.append(f"Global summary: {state['summary']}")
    elif global_text:
        m2 = re.search(r"\[SUMMARY\]\s*(.*?)(?:\n\[|$)", global_text, re.IGNORECASE | re.DOTALL)
        if m2:
            ctx_lines.append("Global summary: " + " ".join(m2.group(1).split()))

    if top_entities:
        ents_str = ", ".join([f"{k} (x{v})" for k, v in top_entities])
        ctx_lines.append(f"Frequent entities: {ents_str}")
    if top_actions:
        acts_str = ", ".join([f"{k} (x{v})" for k, v in top_actions])
        ctx_lines.append(f"Frequent actions: {acts_str}")
    if top_tags:
        tags_str = ", ".join([f"{k} (x{v})" for k, v in top_tags])
        ctx_lines.append(f"Frequent tags: {tags_str}")

    return {
        "likely_type": likely_type,
        "context_text": "\n".join(ctx_lines) if ctx_lines else global_text
    }

# ----------------------------
# Strategy 2 pipeline
# ----------------------------
def chunk_bounds(duration_sec, chunk_sec):
    n = math.ceil(duration_sec / chunk_sec)
    return [(i * chunk_sec, min((i + 1) * chunk_sec, duration_sec)) for i in range(n) if min((i + 1) * chunk_sec, duration_sec) > i * chunk_sec]

def _filesize(path: str) -> str:
    try:
        sz = os.path.getsize(path)
        for unit in ["B", "KB", "MB", "GB", "TB"]:
            if sz < 1024.0:
                return f"{sz:,.2f} {unit}"
            sz /= 1024.0
    except Exception:
        return "n/a"
    return "n/a"

In [11]:
def process_strategy2(video_path: str):
    t0 = time.perf_counter()
    assert os.path.isfile(video_path), f"Video not found: {video_path}"
    LOGGER.info(f"Opening video: {video_path}")

    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    fps = float(vr.get_avg_fps()); total_frames = len(vr)
    duration = total_frames / fps if fps > 0 else 0.0
    width, height = vr[0].shape[1], vr[0].shape[0]
    LOGGER.info(f"Video | fps={fps:.3f} frames={total_frames} dur={duration:.3f}s res={width}x{height}")

    # outputs
    base = os.path.splitext(os.path.basename(video_path))[0]
    stamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_name = f"{base}_{stamp}.csv"
    meta_name = f"{base}_{stamp}_metadata.txt"
    ctx_name = f"{base}_{stamp}_context.json"

    # 3) Global description (whole video sampling)
    LOGGER.info("Sampling frames for GLOBAL description...")
    global_px, global_n, _, _, _ = load_video_pixel_values(
        video_path,
        bound=(0, duration),
        input_size=INPUT_SIZE,
        max_num=TILE_MAX_NUM,
        num_segments=GLOBAL_SAMPLE_SEGMENTS,
        frames_per_segment=GLOBAL_FRAMES_PER_SEGMENT
    )
    LOGGER.info(f"Global sampling: frames={global_n}, segments={GLOBAL_SAMPLE_SEGMENTS}, fps={fps:.3f}")

    LOGGER.info("Generating GLOBAL description...")
    global_text = chat_with_images(global_px, GLOBAL_PROMPT)

    # Init context state
    context_state = init_context_state()
    # Try extracting quick type/summary from global_text
    m_type = re.search(r"\[TYPE\]\s*(.*)", global_text, re.IGNORECASE)
    m_sum = re.search(r"\[SUMMARY\]\s*(.*?)(?:\n\[|$)", global_text, re.IGNORECASE | re.DOTALL)
    if m_type: context_state["video_type"] = " ".join(m_type.group(1).split())
    if m_sum: context_state["summary"] = " ".join(m_sum.group(1).split())

    # 1) Divide into 10s scenes
    LOGGER.info("Computing 10s scene bounds...")
    bounds = chunk_bounds(duration, SCENE_SECONDS)
    total_scenes = len(bounds)
    LOGGER.info(f"{total_scenes} scene(s) planned.")

    # CSV rows
    rows = []

    # 4 & 5) Loop over scenes: extract & update context
    for i, (start_s, end_s) in enumerate(bounds, start=1):
        pct = (i / total_scenes) * 100 if total_scenes else 100
        LOGGER.info(f"[{i}/{total_scenes}] ({pct:4.1f}%) Scene {seconds_to_tc(start_s)}–{seconds_to_tc(end_s)}")

        # Build evolving global context text
        confronted = confront_facts(context_state, global_text)
        evolving_context_text = confronted["context_text"]

        try:
            # Sample frames for this scene (1–2 per segment as configured)
            px, sampled, fps_local, w, h = load_video_pixel_values(
                video_path,
                bound=(start_s, end_s),
                input_size=INPUT_SIZE,
                max_num=TILE_MAX_NUM,
                num_segments=SCENE_SEGMENTS,
                frames_per_segment=SCENE_FRAMES_PER_SEGMENT
            )
            LOGGER.info(f"  • Scene frames sampled: {sampled} (segments={SCENE_SEGMENTS}, frames/seg={SCENE_FRAMES_PER_SEGMENT})")

            # Build scene prompt using evolving global context
            scene_prompt = build_scene_prompt(
                global_context_text=evolving_context_text,
                scene_id=i,
                start_s=start_s,
                end_s=end_s,
                video_name=base,
                start_tc=seconds_to_tc(start_s),
                end_tc=seconds_to_tc(end_s)
            )

            # Generate RAG Scene Card
            scene_text = chat_with_images(px, scene_prompt)

            # Update context from this scene
            update_context_from_scene(context_state, scene_text, global_text_hint=global_text)

            # Row
            row = {
                "video_name": base,
                "video_path": os.path.abspath(video_path),
                "scene_id": i,
                "start_sec": round(float(start_s), 3),
                "end_sec": round(float(end_s), 3),
                "start_timecode": seconds_to_tc(start_s),
                "end_timecode": seconds_to_tc(end_s),
                "scene_duration_sec": round(float(end_s - start_s), 3),
                "fps": fps_local,
                "width": w,
                "height": h,
                "sampled_frames_for_scene": sampled,
                "generated_text": scene_text
            }
            rows.append(row)
            LOGGER.info(f"  • Scene {i} processed.")

        except Exception as e:
            LOGGER.exception(f"  × Scene {i} failed: {e}")
            rows.append({
                "video_name": base,
                "video_path": os.path.abspath(video_path),
                "scene_id": i,
                "start_sec": round(float(start_s), 3),
                "end_sec": round(float(end_s), 3),
                "start_timecode": seconds_to_tc(start_s),
                "end_timecode": seconds_to_tc(end_s),
                "scene_duration_sec": round(float(end_s - start_s), 3),
                "fps": fps, "width": width, "height": height,
                "sampled_frames_for_scene": 0,
                "generated_text": f"[ERROR] {e}"
            })

    # 6) Save results
    # CSV
    LOGGER.info("Writing CSV...")
    fieldnames = list(rows[0].keys()) if rows else [
        "video_name","video_path","scene_id","start_sec","end_sec",
        "start_timecode","end_timecode","scene_duration_sec","fps","width","height",
        "sampled_frames_for_scene","generated_text"
    ]
    with open(csv_name, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames); writer.writeheader()
        for r in rows: writer.writerow(r)
    LOGGER.info(f"CSV saved: {csv_name} ({_filesize(csv_name)})")

    # Context JSON (frequencies + confronted facts snapshot)
    confronted_final = confront_facts(context_state, global_text)
    ctx_payload = {
        "video_name": base,
        "created_at": stamp,
        "scenes_seen": context_state["scenes_seen"],
        "video_type": context_state["video_type"],
        "summary": context_state["summary"],
        "confronted": confronted_final,
        "entities_freq": context_state["entities_freq"],
        "entity_types": context_state["entity_types"],
        "actions_freq": context_state["actions_freq"],
        "tags_freq": context_state["tags_freq"],
        "ocr_vocab_top": sorted(context_state["ocr_vocab"].items(), key=lambda x: (-x[1], x[0]))[:100]
    }
    with open(ctx_name, "w", encoding="utf-8") as f:
        json.dump(ctx_payload, f, ensure_ascii=False, indent=2)
    LOGGER.info(f"Context JSON saved: {ctx_name} ({_filesize(ctx_name)})")

    # Metadata TXT (technical + model-global + confronted facts)
    technical_meta = {
        "file_name": os.path.basename(video_path),
        "abs_path": os.path.abspath(video_path),
        "created_at": stamp,
        "video_length_sec": round(duration, 3),
        "fps": fps,
        "width": width,
        "height": height,
        "num_scenes_10s": len(bounds),
        "scene_seconds": SCENE_SECONDS,
        "global_sample_frames": global_n
    }
    top_ents_str = ", ".join([f"{k} (x{v})" for k, v in sorted(context_state["entities_freq"].items(), key=lambda x: (-x[1], x[0]))[:20]])
    top_acts_str = ", ".join([f"{k} (x{v})" for k, v in sorted(context_state["actions_freq"].items(), key=lambda x: (-x[1], x[0]))[:20]])
    top_tags_str = ", ".join([f"{k} (x{v})" for k, v in sorted(context_state["tags_freq"].items(), key=lambda x: (-x[1], x[0]))[:20]])

    meta_text = [
        "# GENERAL VIDEO METADATA",
        "",
        "## Technical",
        json.dumps(technical_meta, indent=2),
        "",
        "## Model-Inferred (Global Sampling)",
        global_text.strip(),
        "",
        "## Aggregated Context (Confronted Facts)",
        f"Likely type: {confronted_final.get('likely_type') or context_state['video_type']}",
        confronted_final.get("context_text", ""),
        "",
        "## Top Entities",
        top_ents_str or "(none)",
        "",
        "## Top Actions",
        top_acts_str or "(none)",
        "",
        "## Top Tags",
        top_tags_str or "(none)",
        ""
    ]
    with open(meta_name, "w", encoding="utf-8") as f:
        f.write("\n".join(meta_text))
    LOGGER.info(f"Metadata TXT saved: {meta_name} ({_filesize(meta_name)})")

    LOGGER.info(f"Done in {time.perf_counter() - t0:.2f}s | CSV: {csv_name} | TXT: {meta_name} | JSON: {ctx_name}")
    return csv_name, meta_name, ctx_name

In [12]:
GLOBAL_PROMPT = (
    "You are analyzing sampled frames from the WHOLE video. Provide:\n"
    "- Video type/genre (e.g., sports/soccer match highlight, vlog, screencast, ad, tutorial, etc.)\n"
    "- High-level summary (4-5 sentences)\n"
    "- Likely action types present (bulleted)\n"
    "- Prominent entities/teams/brands (bulleted)\n"
    "- Visual style cues (e.g., broadcast overlay, handheld, studio)\n"
    "Return sections: [TYPE], [SUMMARY], [ACTIONS], [ENTITIES], [STYLE]."
)

In [13]:
# ----------------------------
# Prompts
# ----------------------------
GLOBAL_PROMPT = (
    "You are analyzing sampled frames from the WHOLE video. Provide:\n"
    "- Video type/genre (e.g., sports/soccer match highlight, vlog, screencast, ad, tutorial, etc.)\n"
    "- High-level summary (4-5 sentences)\n"
    "Return sections: [TYPE], [SUMMARY]"
)

def build_scene_prompt(global_context_text: str,
                       scene_id: int,
                       start_s: float,
                       end_s: float,
                       video_name: str,
                       start_tc: str,
                       end_tc: str) -> str:
    """
    RAG-friendly SCENE CARD (text, not JSON), includes global context and asks for
    Description, OCR, Entities, Actions (chronological).
    """
    return f"""
You will receive frames from a ~10-second SCENE of a video.
Use the GLOBAL CONTEXT to disambiguate when helpful, but do not contradict the visible frames.

GLOBAL CONTEXT (aggregated; may include inferred type/summary/entities/actions):
{global_context_text.strip()}

TASK:
Return a RAG SCENE CARD in plain text (no code fences). Keep it factual and concise.

RAG SCENE CARD
---
Meta:
- video_name: {video_name}
- scene_id: {scene_id}
- start_sec: {start_s:.3f}
- end_sec: {end_s:.3f}
- start_timecode: {start_tc}
- end_timecode: {end_tc}

Description:
- description: <4-5 lines of who/what/where and context>
- actions_chronological:
  - <t≈ +0-2s: action phrase>
  - <t≈ +2-4s: action phrase>
  - <...>

OnScreenText:
- lines:
  - <exact OCR line 1>
  - <exact OCR line 2>
  - ...

Entities:
- items:
  - type: <person/team/logo/brand/location/object/number/other>
    name_or_value: <best guess or exact text>
    attributes: [<short attrs like jersey #, color, role, number>]
  - ...

Tags:
- scene_tags: [<short keywords like teams, brands, numbers, jersey ids, location>, ...]

Shot:
- camera_motion: [<pan/tilt/handheld/static/zoom>, ...]
- camera_angle: [<wide/close-up/top-down>, ...]
- cuts_or_transitions: [<if any>, ...]

Confidence:
- overall: <0.0-1.0>
- ocr: <0.0-1.0>
- entity_detection: <0.0-1.0>

NOTES:
- For OnScreenText, list ALL legible text exactly as written (preserve case).
- Prefer short bullets for actions/tags/entities; keep chronology coarse (0–2s, 2–4s, etc.).
- If unknown or not visible, write 'unknown' or use empty lists [].
- Return ONLY the card content above (no extra commentary).
""".strip()

In [ ]:
process_strategy2(VIDEO_PATH)

2025-09-13 15:42:42 | INFO    | Opening video: documentary_480p.mp4
2025-09-13 15:42:42 | INFO    | Video | fps=25.000 frames=21676 dur=867.040s res=854x358
2025-09-13 15:42:42 | INFO    | Sampling frames for GLOBAL description...
2025-09-13 15:42:44 | INFO    | Global sampling: frames=16, segments=16, fps=25.000
2025-09-13 15:42:44 | INFO    | Generating GLOBAL description...
2025-09-13 15:42:44 | INFO    | chat_with_images | CUDA | tensor=(16, 3, 448, 448)
2025-09-13 15:42:44 | INFO    | Prompt length: 344 chars
2025-09-13 15:42:44 | INFO    | >>> Generation started...


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


[TYPE]  
Documentary/Nature Film  

[SUMMARY]  
The video showcases a safari adventure in the wild, capturing scenes of vehicles navigating through open grasslands. It features close-ups of cheetahs interacting with humans and highlights various moments of wildlife observation at sunset. The footage emphasizes exploration and encounters with nature, focusing on both human activities and animal behavior in their natural habitat.2025-09-13 15:42:52 | INFO    | 
>>> Generation finished in 8.08s (431 chars).
2025-09-13 15:42:52 | INFO    | Computing 10s scene bounds...
2025-09-13 15:42:52 | INFO    | 87 scene(s) planned.
2025-09-13 15:42:52 | INFO    | [1/87] ( 1.1%) Scene 00:00:00.000–00:00:10.000
2025-09-13 15:42:53 | INFO    |   • Scene frames sampled: 10 (segments=10, frames/seg=1)
2025-09-13 15:42:53 | INFO    | chat_with_images | CUDA | tensor=(10, 3, 448, 448)
2025-09-13 15:42:53 | INFO    | Prompt length: 2135 chars
2025-09-13 15:42:53 | INFO    | >>> Generation started...


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


RAG SCENE CARD
---
Meta:
- video_name: documentary_480p
- scene_id: 1
- start_sec: 0.000
- end_sec: 10.000 
- start_timecode: 00:00:00.000  
- end_timecode: 01:00:10.001  

Description:
- description: Aerial view of a dirt road in an open grassland, with sparse vegetation and distant hills under a clear sky.
- actions_chronological:
  - <t≈ +0-2s>: Camera pans over the landscape from above
  - <t≈+2-4s>: The camera continues to move forward along the road

OnScreenText:
- lines:
  - "100 YEARS AGO... THERE WERE APPROXIMATELY 65,000 CHEETAHS ON THIS PLANET."

Entities:
- items:
  - type: text
    name_or_value: "100 YEARS ago.. there were approximately 65,0o0 cheetahs on this planet."
    attributes: []

Tags:
- scene_tags: [nature, wildlife, safari]

Shot:
- camera_motion: [aerial]
- camera_angle: [top-down]
- cuts_or_transitions: []

Confidence:
- overall: 0.9
- ocr: 0.95
- entity_detection: 0.85
2025-09-13 15:43:20 | INFO    | 
>>> Generation finished in 27.02s (910 chars).
2025-09-1

...

RAG SCENE CARD
---
Meta:
- video_name: documentary_480p
- scene_id: 85
- start_sec: 840.000
- end_sec: 850.000 
- start_timecode: 00:14:00.000  
- end_timecode: 01:14:10.000  

Description:
- description: Aerial view of a grassy landscape with sparse vegetation, likely part of a safari setting.
- actions_chronological:
  - <t≈ +0-2s>: Camera pans over the landscape from above.
  - <t≈+2-4s>: No significant action changes observed.

OnScreenText:
- lines:
  - EXECUTIVE PRODUCER ISAAC WRIGHT
  - DIRECTOR OF PHOTOGRAPHY RICARDO NEIVA
  - ADDITIONAL CAMERA ISAAC WRIGHT JACK ZAKRAJEK DRONE

Entities:
- items:
  - type: object
    name_or_value: tree
    attributes: [sparse branches]
  - type: text
    name_or_value:** Executive Producer**
    attributes: []
  - type: text 
    name_or_value:** Director of Photography** 
    attributes: []

Tags:
- scene_tags: [nature, wildlife, safari]

Shot:
- camera_motion: [pan]
- camera_angle: [top-down]
- cuts_or_transitions: []

Confidence:
- overall:

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


RAG SCENE CARD
---
Meta:
- video_name: documentary_480p
- scene_id: 86
- start_sec: 850.000
- end_sec: 860.000 
- start_timecode: 00:14:10.000  
- end_timecode: 01:14:20.000  

Description:
- description: A cheetah is perched on a tree in tall grasslands during sunset, likely part of a wildlife observation segment.
- actions_chronological:
  - <t≈ +0-2s>: Cheetah remains stationary while looking around.
  - <t≈+2-4s>: No significant action changes observed.

OnScreenText:
- lines:
  - SOUND DESIGN RICARDO NEIVA
  - SOUND MIX DAN ARBUSCH LOYAL SOUND

Entities:
- items:
  - type: person/team/logo/brand/location/object
    name_or_value: unknown
    attributes: []
  - type: text
    name_or_value:**SOUND DESIGN** RICARDO NEIVALOGO
    attributes: []

Tags:
- scene_tags: [nature, wildlife, safari]

Shot:
- camera_motion: [static]
- camera_angle: [top-down]
- cuts_or_transitions: []

Confidence:
- overall: 0.9
- ocr: 0.95
- entity_detection: 0.7
2025-09-13 16:22:00 | INFO    | 
>>> Generati

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


RAG SCENE CARD
---
Meta:
- video_name: documentary_480p
- scene_id: 87
- start_sec: 860.000
- end_sec: 867.040
- start_timecode: 00:14:20.000 
- end_timecode: 01:14:27.040

Description:
- description: A cheetah is perched on a tree branch in grasslands during sunset, observing its surroundings.
- actions_chronological:
  - <t≈ +0-2s>: Cheetah remains stationary while looking around.
  - <t≈+2-4s>: No significant action changes observed.

OnScreenText:
- lines:
  - CC COLORIST DELANO BORGES ROIRO
  - SPECIAL THANKS TO RICH ZAKRAUSEK AMY ZAKRAUSEK DYLAN ZAKRAUSEK

Entities:
- items:
  - type: animal
    name_or_value: cheetah
    attributes: []
  - type: text
    name_or_value:
      - "CC COLORIST DELANO BORGERO"
